## 折衷查表:
### 用 Single Error Correction 解 Double Error Correction (A 取Double_AWE的 A)

#### 1.取 A的bits $\approx$ N的bits $\Rightarrow$ A = 3349 (12bits), N的範圍: 0 ~ 4095 (12bits)

### Define function
* (Not including 2's complement range)

In [1]:
import math
import numpy as np


def decimal_to_binary(decimal_number):
    if decimal_number == 0:
        return "0"
    # 處理正數：直接轉換
    if decimal_number > 0:
        return bin(decimal_number)[2:]  # 移除 "0b" 前綴
       # 處理負數：自動計算所需位元數（2's complement）
    abs_value = abs(decimal_number)
    bits = abs_value.bit_length() + 1  # 負數多一個符號位元
    two_complement_value = (1 << bits) + decimal_number
    
    return bin(two_complement_value)[2:]  # 移除 "0b" 前綴

def bits_needed(number):
    return number.bit_length()


def bits_range(bits, signed=False):     ## 可選擇設定 有號數 或 無號數
    if bits <= 0:
        return "位元數必須為正整數！"
    
    if signed:
        # 二補數範圍: -2^(bits-1) 到 2^(bits-1)-1
        min_value = -(2**(bits - 1))
        max_value = 2**(bits - 1) - 1
    else:
        # 無符號範圍: 0 到 2^bits - 1
        min_value = 0
        max_value = 2**bits - 1

    return min_value, max_value

## 把重複的 AWE 去除
def remove_duplicates(input_list):
    unique_list = []
    for number in input_list:
        if number not in unique_list:
            unique_list.append(number)
    return unique_list


def get_sign(number):
    return 1 if number > 0 else -1 if number < 0 else 0


print("一個2補數所需要的位元數",bits_needed(120))
print("一bits數可表示的2補數的範圍",bits_range(bits_needed(120)))
print("一個2補數表達為二進制為:",decimal_to_binary(-8))

一個2補數所需要的位元數 7
一bits數可表示的2補數的範圍 (0, 127)
一個2補數表達為二進制為: 11000


### 查找表範例

In [2]:
def create_mapping(source_list, address_map):
    """
    建立值到位址和位址到值的雙向映射。
    若 address_map 中某位址沒有對應值，則該位址的值為 0。
    
    參數:
    source_list (list): 原始資料列表。
    address_map (list): 位址映射列表。
    
    回傳:
    tuple: (值到位址的映射, 位址到值的映射)
    """
    # 初始化值到位址的映射
    value_to_address = {value: addr for value, addr in zip(source_list, address_map)}
    
    # 初始化位址到值的映射，未指定的位址預設為 0
    max_address = max(address_map)  # 確保涵蓋所有位址
    address_to_value = {addr: 0 for addr in range(max_address + 1)}  # 預設為 0
    for value, addr in zip(source_list, address_map):
        address_to_value[addr] = value  # 更新有指定值的位址

    return value_to_address, address_to_value

# 測試數據
source_list = [10, 20, 30]
address_map = [2, 0, 4]

# 建立映射
value_to_address, address_to_value = create_mapping(source_list, address_map)

# 顯示結果
print("值到位址的映射:", value_to_address)
print("位址到值的映射:", address_to_value)


# 測試數據
source_list = [10, 20, 30, 40]
address_map = [2, 0, 3, 1]

# 建立映射
value_to_address, address_to_value = create_mapping(source_list, address_map)

# 測試查詢
print("值到位址:", value_to_address)  # 查詢 10 -> 2, 20 -> 0
print("位址到值:", address_to_value)  # 查詢 2 -> 10, 0 -> 20

# 範例查詢
value = 10
address = 2
print(f"輸入值 {value}, 對應位址: {value_to_address[value]}")
print(f"輸入位址 {address}, 對應值: {address_to_value[address]}")

值到位址的映射: {10: 2, 20: 0, 30: 4}
位址到值的映射: {0: 20, 1: 0, 2: 10, 3: 0, 4: 30}
值到位址: {10: 2, 20: 0, 30: 3, 40: 1}
位址到值: {0: 20, 1: 40, 2: 10, 3: 30}
輸入值 10, 對應位址: 2
輸入位址 2, 對應值: 10


## 以12bits為例:

In [3]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data1 = (1 << 12) - 1    ## 
data2 = (1 << 12) - 15
data3 = (1 << 12) - 17
AN = 3349
input1 = data1*AN
input2 = data2*AN
input3 = data3*AN
bits1 = bits_needed(input1)
bits2 = bits_needed(input2)
bits3 = bits_needed(input3)
print("inputs1的位元數:", bits1)
print("inputs2的位元數:", bits2)
print("inputs3的位元數:", bits3)
print(data1)

inputs1的位元數: 24
inputs2的位元數: 24
inputs3的位元數: 24
4095


### 產生1-bit AWE (Arithmetic-Weight Error)
* 並產生 Delta Q 的 list

In [4]:
AWE1_1=[]
e1_1=0     ## 錯誤數量
e1_data1=[]
h = [];   #位置
for i in range(0, bits1):
    e1_data1.insert(e1_1 , input1 + 2**i)
    AWE1_1.insert(e1_1, 2**i)
    e1_1 = e1_1 + 1
    e1_data1.insert(e1_1 , input1 - 2**i)
    AWE1_1.insert(e1_1, -2**i)
    e1_1 = e1_1 + 1
    h.insert(i,i+1)

AWE1_2=[]
e1_2=0     ## 錯誤數量
e1_data2=[]
for i in range(0, bits2):
    e1_data2.insert(e1_2 , input2 + 2**i)
    AWE1_2.insert(e1_2, 2**i)
    e1_2 = e1_2 + 1
    e1_data2.insert(e1_2 , input2 - 2**i)
    AWE1_2.insert(e1_2, -2**i)
    e1_2 = e1_2 + 1
    ### h.insert(i,i+1)
    
AWE1_3=[]
e1_3=0     ## 錯誤數量
e1_data3=[]
for i in range(0, bits3):
    e1_data3.insert(e1_3 , input3 + 2**i)
    AWE1_3.insert(e1_3, 2**i)
    e1_3 = e1_3 + 1
    e1_data3.insert(e1_3 , input3 - 2**i)
    AWE1_3.insert(e1_3, -2**i)
    e1_3 = e1_3 + 1
    ### h.insert(i,i+1)

h_awe = [val for num in h for val in (num, -num)]
deltaQ1 = [(x // AN) - data1 for x in e1_data1]
deltaQ2 = [(x // AN) - data2 for x in e1_data2]
deltaQ3 = [(x // AN) - data3 for x in e1_data3]
print(deltaQ1)
print(deltaQ2)
print(deltaQ3)
## print(deltaQ1 == deltaQ2)
## print(deltaQ1 == deltaQ3)
print("不同輸入資料的deltaQ是否相等:",deltaQ3 == deltaQ2 == deltaQ1)
print(np.size(deltaQ1))

### 確定了Delta Q 的 list 是唯一 ###

print("產生1-bit AWE:", AWE1_1)
print("不同輸入資料的AWE是否相等:",AWE1_1 == AWE1_2 == AWE1_3)
print(h_awe)
print(np.size(AWE1_1))

[0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 1, -2, 2, -3, 4, -5, 9, -10, 19, -20, 39, -40, 78, -79, 156, -157, 313, -314, 626, -627, 1252, -1253, 2504, -2505]
[0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 1, -2, 2, -3, 4, -5, 9, -10, 19, -20, 39, -40, 78, -79, 156, -157, 313, -314, 626, -627, 1252, -1253, 2504, -2505]
[0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 0, -1, 1, -2, 2, -3, 4, -5, 9, -10, 19, -20, 39, -40, 78, -79, 156, -157, 313, -314, 626, -627, 1252, -1253, 2504, -2505]
不同輸入資料的deltaQ是否相等: True
48
產生1-bit AWE: [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608]
不同輸入資料的AWE是否相等: True
[1, -1, 2, -2, 3, -3, 4, -

In [5]:
def transform_h(n):
    return 2**(abs(n)-1) if n > 0 else -2**(abs(n)-1) 

print(transform_h(16))

32768


### 產生SEC的模數:

In [6]:
SEC_R = []
AWE1_R= []
for m in range(0,e1_1):
    mod = e1_data1[m] % AN
    SEC_R.append(mod)

for n in range(0,e1_1):
    mod_awe = AWE1_3[n] % AN
    AWE1_R.append(mod_awe)


print("SEC的模數:\n",SEC_R)
print("AWE1的模數:\n",SEC_R)
print("AWE1的模數是否等於SEC模數:", SEC_R == AWE1_R)
print("SEC的大小:",np.size(SEC_R))

SEC的模數:
 [1, 3348, 2, 3347, 4, 3345, 8, 3341, 16, 3333, 32, 3317, 64, 3285, 128, 3221, 256, 3093, 512, 2837, 1024, 2325, 2048, 1301, 747, 2602, 1494, 1855, 2988, 361, 2627, 722, 1905, 1444, 461, 2888, 922, 2427, 1844, 1505, 339, 3010, 678, 2671, 1356, 1993, 2712, 637]
AWE1的模數:
 [1, 3348, 2, 3347, 4, 3345, 8, 3341, 16, 3333, 32, 3317, 64, 3285, 128, 3221, 256, 3093, 512, 2837, 1024, 2325, 2048, 1301, 747, 2602, 1494, 1855, 2988, 361, 2627, 722, 1905, 1444, 461, 2888, 922, 2427, 1844, 1505, 339, 3010, 678, 2671, 1356, 1993, 2712, 637]
AWE1的模數是否等於SEC模數: True
SEC的大小: 48


### 建立只能做SEC的LUT: Mapping (+-)location to (SEC_R, deltaQ1)

In [7]:
## mapping (+-)location to R
h_to_R, R_to_h = create_mapping(h_awe, SEC_R) 
## mapping (+-)location to deltaQ
h_to_deltaQ, deltaQ_to_h = create_mapping(h_awe, deltaQ1)


### 產生2-bit AWE (Arithmetic-Weight Error)

In [8]:
AWE_2=[]
e2=0     ## 錯誤數量
p=0      ## 錯誤位置
e2_data=[]
for i in range(0, bits1):
    for j in range(i+1, bits1):
        e2_data.insert(e2 , (input1 + 2**i + 2**j))
        AWE_2.insert(e2 , (2**i + 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 + 2**i - 2**j))
        AWE_2.insert(e2 , (2**i - 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i + 2**j))
        AWE_2.insert(e2 , (-2**i + 2**j))                
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i -2**j))
        AWE_2.insert(e2 , (-2**i - 2**j))
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        p = p + 1


print(e2)
print(p)
print("產生2-bits AWE:",AWE_2)
print("產生2-bits 錯誤資料:",e2_data)

第 0 筆錯誤資料: 13714158
第 0 筆算術錯誤: 3
第 1 筆錯誤資料: 13714154
第 1 筆算術錯誤: -1
第 2 筆錯誤資料: 13714156
第 2 筆算術錯誤: 1
第 3 筆錯誤資料: 13714152
第 3 筆算術錯誤: -3
第 4 筆錯誤資料: 13714160
第 4 筆算術錯誤: 5
第 5 筆錯誤資料: 13714152
第 5 筆算術錯誤: -3
第 6 筆錯誤資料: 13714158
第 6 筆算術錯誤: 3
第 7 筆錯誤資料: 13714150
第 7 筆算術錯誤: -5
第 8 筆錯誤資料: 13714164
第 8 筆算術錯誤: 9
第 9 筆錯誤資料: 13714148
第 9 筆算術錯誤: -7
第 10 筆錯誤資料: 13714162
第 10 筆算術錯誤: 7
第 11 筆錯誤資料: 13714146
第 11 筆算術錯誤: -9
第 12 筆錯誤資料: 13714172
第 12 筆算術錯誤: 17
第 13 筆錯誤資料: 13714140
第 13 筆算術錯誤: -15
第 14 筆錯誤資料: 13714170
第 14 筆算術錯誤: 15
第 15 筆錯誤資料: 13714138
第 15 筆算術錯誤: -17
第 16 筆錯誤資料: 13714188
第 16 筆算術錯誤: 33
第 17 筆錯誤資料: 13714124
第 17 筆算術錯誤: -31
第 18 筆錯誤資料: 13714186
第 18 筆算術錯誤: 31
第 19 筆錯誤資料: 13714122
第 19 筆算術錯誤: -33
第 20 筆錯誤資料: 13714220
第 20 筆算術錯誤: 65
第 21 筆錯誤資料: 13714092
第 21 筆算術錯誤: -63
第 22 筆錯誤資料: 13714218
第 22 筆算術錯誤: 63
第 23 筆錯誤資料: 13714090
第 23 筆算術錯誤: -65
第 24 筆錯誤資料: 13714284
第 24 筆算術錯誤: 129
第 25 筆錯誤資料: 13714028
第 25 筆算術錯誤: -127
第 26 筆錯誤資料: 13714282
第 26 筆算術錯誤: 127
第 27 筆錯誤資料: 13714026
第 27 筆算術錯誤: -129
第 28 筆

In [9]:
DEC_R = []
for m in range(0,e2):
    mod = e2_data[m] % AN
    DEC_R.append(mod)

print("DEC_R的模數:\n",DEC_R)
print(np.size(DEC_R))

DEC_R的模數:
 [3, 3348, 1, 3346, 5, 3346, 3, 3344, 9, 3342, 7, 3340, 17, 3334, 15, 3332, 33, 3318, 31, 3316, 65, 3286, 63, 3284, 129, 3222, 127, 3220, 257, 3094, 255, 3092, 513, 2838, 511, 2836, 1025, 2326, 1023, 2324, 2049, 1302, 2047, 1300, 748, 2603, 746, 2601, 1495, 1856, 1493, 1854, 2989, 362, 2987, 360, 2628, 723, 2626, 721, 1906, 1445, 1904, 1443, 462, 2889, 460, 2887, 923, 2428, 921, 2426, 1845, 1506, 1843, 1504, 340, 3011, 338, 3009, 679, 2672, 677, 2670, 1357, 1994, 1355, 1992, 2713, 638, 2711, 636, 6, 3347, 2, 3343, 10, 3343, 6, 3339, 18, 3335, 14, 3331, 34, 3319, 30, 3315, 66, 3287, 62, 3283, 130, 3223, 126, 3219, 258, 3095, 254, 3091, 514, 2839, 510, 2835, 1026, 2327, 1022, 2323, 2050, 1303, 2046, 1299, 749, 2604, 745, 2600, 1496, 1857, 1492, 1853, 2990, 363, 2986, 359, 2629, 724, 2625, 720, 1907, 1446, 1903, 1442, 463, 2890, 459, 2886, 924, 2429, 920, 2425, 1846, 1507, 1842, 1503, 341, 3012, 337, 3008, 680, 2673, 676, 2669, 1358, 1995, 1354, 1991, 2714, 639, 2710, 635, 12, 3

In [10]:
total_AWE = AWE1_1 + AWE_2
e_total = e1_data1 + e2_data
R_total = SEC_R + DEC_R
print("還未去除重複error data的數量:",np.size(e_total))
print("還未去除重複餘數的數量:", np.size(R_total))

## 把重複的 AWE 及 error_data 去除
AWE = remove_duplicates(total_AWE)
new_e_total = remove_duplicates(e_total)
new_R_total = remove_duplicates(R_total)
error_total_count = np.size(new_e_total)
print("已去除重複error data的數量:",error_total_count)
print("已去除重複餘數的數量:",np.size(new_R_total))
print("已去除重複AWE的數量:",np.size(AWE))
print("所有的AWE為:\n",AWE)
print("所有算術錯誤的資料:\n", new_e_total)
print("所有的R為:\n",new_R_total )

# 寫入 txt 檔，每個數字一行
with open("test_vectors.txt", "w") as f:
    for number in new_e_total:
        f.write(f"{number}\n")

還未去除重複error data的數量: 1152
還未去除重複餘數的數量: 1152
已去除重複error data的數量: 1062
已去除重複餘數的數量: 1062
已去除重複AWE的數量: 1062
所有的AWE為:
 [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 3, -3, 5, -5, 9, -7, 7, -9, 17, -15, 15, -17, 33, -31, 31, -33, 65, -63, 63, -65, 129, -127, 127, -129, 257, -255, 255, -257, 513, -511, 511, -513, 1025, -1023, 1023, -1025, 2049, -2047, 2047, -2049, 4097, -4095, 4095, -4097, 8193, -8191, 8191, -8193, 16385, -16383, 16383, -16385, 32769, -32767, 32767, -32769, 65537, -65535, 65535, -65537, 131073, -131071, 131071, -131073, 262145, -262143, 262143, -262145, 524289, -524287, 524287, -524289, 1048577, -1048575, 1048575, -1048577, 2097153, -2097151, 2097151, -2097153, 4194305, -4194303, 4194303, -4194305, 8388609, -83886

### 所有AWE數量: $4*C^{bits}_{2}$ $+$ $2*C^{bits}_{1}$  $-$ $2*(bits-1)$ - $2*(bits-2)$
* $2*(bits-1)$: 所有的 single AWE都可以分解成已重複的 double AWE, 除了MSB例外
* EX: $(+,0)$ 只能分解成 $(0,++)$ , $(-,0)$ 只能分解成 $(0,--)$
* $2*(bits-2)$: double AWE重複的個數
* EX: $(+,0,-)$ = $(0,+,+)$ and $(-,0,+)$ = $(0,-,-)$

### Trade-off Algorithm: 2025/02/20
* 此解出來的數字為位置及正負號 EX: $-1$ 代表 $-2^0$ , $-5$ 代表 $-2^4$ 以此類推

In [11]:
### 給一個R, 找錯誤位置h1及h2(包含錯誤正負號)
###
import math
import numpy as np

h1 = 0; h2 = 0; h_temp = 0
possible_h1 = []; possible_h2 = []; possible_AWE = []
output = []; possible_SE_h1=[]


for R, error in zip(new_R_total, new_e_total): 
    found = False                                  # 旗標，用來標示是否要跳出 for H 循環 
    for H in range(0,bits1):                              
        for s in range(-1,2,2):                       
            h1 = s*(H+1)
            R1 = h_to_R[h1]
            R2 = R - R1
            if R2 ==0:
                print(h1)
                # print(s * 2**(abs(h1)-1))
                # output.append((error // AN) - h_to_deltaQ[h1])
                possible_SE_h1.append(s * 2**(abs(h1)-1))
                output.append((error - (s * 2**(abs(h1)-1))) // AN)
                found = True
                break
            elif R2 < 0:
                R2 = AN + R2                    
            h2 = R_to_h[R2]                  
            if (h2 != 0):
                s2 = get_sign(h2)                     
                possible_h1.append(h1)              # 若找不到對應的h2,h2定為0(函數內容已定義)
                possible_h2.append(h2)              # 代表錯誤可能是三個或以上(multiple errors)
                output.append((error - (s*2**(abs(h1)-1)) - (s2 * 2**(abs(h2)-1))) // AN)
                found = True
                break
        if found:
            break
                            
print(possible_h1)
print(possible_h2)
print("更正後的值:",output)
print("更正後的值是否和原data相同:", data1 == output.pop())
# print(data1)
print(np.size(possible_h1))
print(np.size(possible_h2))
print("輸入的 error data的總數是否和經過演算法後相同:", error_total_count == (np.size(possible_SE_h1) + np.size(possible_h1)))
# print(possible_SE_h1)

-1
[-1, 1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14, 15, -15, 16, -16, 17, -17, 18, -18, 19, -19, 20, -20, 21, -21, 22, -22, 23, -23, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, -2, -2, 2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, -3, -3, 3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3,

#### 透過演算法發現大部分的 Single AWE 都會被拆成 Double AWE, 因此 deltaQ-table 應該不需要 

In [12]:
AWE_1_from_h = [2**(abs(n)-1) if n > 0 else -2**(abs(n)-1) for n in possible_h1]
AWE_2_from_h = [2**(abs(n)-1) if n > 0 else -2**(abs(n)-1) for n in possible_h2]
Total_AWE_from_h = [a+b for a, b in zip(AWE_1_from_h, AWE_2_from_h)]
AWE_compared = [item for item in AWE if item not in possible_SE_h1]



print("h1的AWE為:",AWE_1_from_h)
print("h2的AWE為:",AWE_2_from_h)
print("解出來的AWE值為:",Total_AWE_from_h)
print("解出來的AWE是否和原AWE相同:",AWE_compared == Total_AWE_from_h)

h1的AWE為: [-1, 1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, -2, -2, 2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, -4, -4, 4, -4, 4, 4, -4, -4, 4, 4, -4, -4, 4, 4, -4, -4, 4, 4, -4, -4, 4, 4, -4, -4, 4, 4

### Trade-off Algorithm: 2025/02/22
* 此解出來的數字為位置及正負號 EX: $-1$ 代表 $-2^0$ , $-5$ 代表 $-2^4$ 以此類推

In [13]:
### 給一個R, 找錯誤位置h1及h2(包含錯誤正負號)
###
import math
import numpy as np

h1 = 0; h2 = 0; h_temp = 0
possible_h1 = []; possible_h2 = []; possible_AWE = []
output = []; possible_SE_h1=[]


for R, error in zip(new_R_total, new_e_total): 
    found = False                                       # 旗標，用來標示是否要跳出 for H 循環 
    for H in range(0,bits1):                              
        for s in range(-1,2,2):                       
            h1 = s*(H+1)
            R1 = h_to_R[h1]
            R2 = R - R1
            if R2 < 0:
                R2 = AN + R2                    
            h2 = R_to_h[R2]                  
            if (h2 != 0):
                s2 = get_sign(h2)                     
                possible_h1.append(h1)                  # 若找不到對應的h2,h2定為0(函數內容已定義)
                possible_h2.append(h2)                  # 代表錯誤可能是三個或以上(multiple errors)
                output.append((error - (s*2**(abs(h1)-1)) - (s2 * 2**(abs(h2)-1))) // AN)
                found = True
                break
        if found:
            break

print(possible_h1)
print(possible_h2)
print("更正後的值:",output)
print("更正後的值是否和原data相同:", data1 == output.pop())
print(np.size(possible_h1))
print(np.size(possible_h2))
print("輸入的 error data的總數是否和經過演算法後相同:", error_total_count == np.size(possible_h1))
# print(possible_SE_h1)

[-1, 1, 1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14, 15, -15, 16, -16, 17, -17, 18, -18, 19, -19, 20, -20, 21, -21, 22, -22, 23, -23, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, -2, -2, 2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, -3, -3, 3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3, 3, -3, -3, 3,

#### 透過演算法發現"所有"的 Single AWE 都可以被拆成 Double AWE, 因此 deltaQ-table 不需要 

In [14]:
AWE_1_from_h = [2**(abs(n)-1) if n > 0 else -2**(abs(n)-1) for n in possible_h1]
AWE_2_from_h = [2**(abs(n)-1) if n > 0 else -2**(abs(n)-1) for n in possible_h2]
Total_AWE_from_h = [a+b for a, b in zip(AWE_1_from_h, AWE_2_from_h)]

print("h1的AWE為:",AWE_1_from_h)
print("h2的AWE為:",AWE_2_from_h)
print("解出來的AWE值為:",Total_AWE_from_h)
print("解出來的AWE是否和原AWE相同:", AWE == Total_AWE_from_h)

h1的AWE為: [-1, 1, 1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, -1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, -2, -2, 2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, 2, 2, -2, -2, -4, -4, 4, -4, 4, 4, -4, -4, 4, 4, -4, -4, 4, 4, -4, -4, 4, 4, -4, -4, 4, 4, -4, -4, 4

####  Summary: 演算法會解出和原本不一樣的錯誤位置,但所代表的AWE值完全相同,故不影響結果

In [15]:
### 給一個R, 找錯誤位置h1及h2(包含錯誤正負號)
###
import math
import numpy as np

h1 = 0; h2 = 0; h_temp = 0
possible_h1 = []; possible_h2 = []; possible_AWE = []
output = []; possible_SE_h1=[]


R = 1170
error = 297837
found = False                                       # 旗標，用來標示是否要跳出 for H 循環 
for H in range(0,bits1):                              
    for s in range(-1,2,2):                       
        h1 = s*(H+1)
        R1 = h_to_R[h1]
        R2 = R - R1
        if R2 < 0:
            R2 = AN + R2                    
        h2 = R_to_h[R2]                  
        if (h2 != 0):
            s2 = get_sign(h2)                     
            possible_h1.append(h1)                  # 若找不到對應的h2,h2定為0(函數內容已定義)
            possible_h2.append(h2)                  # 代表錯誤可能是三個或以上(multiple errors)
            output.append((error - (s*2**(abs(h1)-1)) - (s2 * 2**(abs(h2)-1))) // AN)
            found = True
            break
    if found:
        break

print(possible_h1)
print(possible_h2)

[]
[]
